# Supplementary Table S2

In [1]:
import h5py
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy.stats as ss

from uoineuro.utils import (calculate_selection_ratio,
                            cohens_d)

%matplotlib inline

## Read in data

In [2]:
ret1_path = os.path.join(os.environ['HOME'], 'fits/uoineuro/tuning/ret1')
ac_path = os.path.join(os.environ['HOME'], 'fits/uoineuro/tuning/ecog/ecog_tuning.h5')

In [3]:
retinas = {
    '20080516_R1': ['cell0_recording0',
                    'cell0_recording2',
                    'cell2_recording0',
                    'cell3_recording0',
                    'cell4_recording0',
                    'cell5_recording0'],
    '20080628_R4': ['cell0_recording0',
                    'cell2_recording0',
                    'cell3_recording0',
                    'cell5_recording0',
                    'cell6_recording0',
                    'cell7_recording0',
                    'cell9_recording0',
                    'cell12_recording0',
                    'cell13_recording0'],
    '20080628_R6': ['cell1_recording0',
                    'cell2_recording0',
                    'cell4_recording0',
                    'cell5_recording0',
                    'cell6_recording0',
                    'cell7_recording0',
                    'cell8_recording0',
                    'cell10_recording0']
}

In [4]:
ret_baseline_srs = []
ret_uoi_srs = []
ret_baseline_r2s = []
ret_uoi_r2s = []
ret_baseline_bics = []
ret_uoi_bics = []

# iterate over retina files
for retina, recordings in retinas.items():
    results = h5py.File(os.path.join(ret1_path, retina + '.h5'), 'r')
    # iterate over recordings
    for cell_recording in recordings:
        # calculate selection ratios
        lasso_strf = results[cell_recording]['lasso/strfs'][:]
        ret_baseline_srs.append(np.count_nonzero(lasso_strf) / lasso_strf.size)
        uoi_strf = results[cell_recording]['uoi_lasso_bic/strfs'][:]
        ret_uoi_srs.append(np.count_nonzero(uoi_strf) / uoi_strf.size)

        # extract predictive performance
        ret_baseline_r2s.append(results[cell_recording]['lasso/r2s_test'][:].max())
        ret_uoi_r2s.append(results[cell_recording]['uoi_lasso_bic/r2s_test'][:].max())

        # extract model parsimony
        ret_baseline_bics.append(results[cell_recording]['lasso/bics'][:].min())
        ret_uoi_bics.append(results[cell_recording]['uoi_lasso_bic/bics'][:].min())

    results.close()
    
ret_baseline_srs = np.array(ret_baseline_srs)
ret_uoi_srs = np.array(ret_uoi_srs)
ret_baseline_r2s = np.array(ret_baseline_r2s)
ret_uoi_r2s = np.array(ret_uoi_r2s)
ret_baseline_bics = np.array(ret_baseline_bics)
ret_uoi_bics = np.array(ret_uoi_bics)

In [5]:
ac_fits = h5py.File(ac_path, 'r')
ac_baseline = ac_fits['lasso']
ac_uoi = ac_fits['uoi_lasso_bic']
ac_baseline_srs = np.mean(calculate_selection_ratio(ac_baseline['tuning_coefs']), axis=0)
ac_baseline_r2s = np.mean(ac_baseline['r2'], axis=0)
ac_baseline_bics = np.mean(ac_baseline['BIC'], axis=0)
ac_uoi_srs = np.mean(calculate_selection_ratio(ac_uoi['tuning_coefs']), axis=0)
ac_uoi_r2s = np.mean(ac_uoi['r2'], axis=0)
ac_uoi_bics = np.mean(ac_uoi['BIC'], axis=0)

## Selection Ratios

In [6]:
print('Retina')
print('------')
baseline_median = np.median(ret_baseline_srs)
uoi_median = np.median(ret_uoi_srs)

print('Baseline: %0.3f +/- %0.3f' %(baseline_median, ss.iqr(ret_baseline_srs) / 2))
print('UoI: %0.3f +/- %0.3f' %(uoi_median, ss.iqr(ret_uoi_srs) / 2))
print('Reduction Factor: ', baseline_median / uoi_median)
print('p-value: ', ss.wilcoxon(ret_baseline_srs, ret_uoi_srs, alternative='greater').pvalue)
print("Cohen's d: ", cohens_d(ret_baseline_srs, ret_uoi_srs))

Retina
------
Baseline: 0.084 +/- 0.020
UoI: 0.017 +/- 0.004
Reduction Factor:  4.984615384615386
p-value:  1.3507974243552756e-05
Cohen's d:  3.0511322004969212


In [7]:
print('AC')
print('--')
baseline_median = np.median(ac_baseline_srs)
uoi_median = np.median(ac_uoi_srs)

print('Baseline: %0.3f +/- %0.3f' %(baseline_median, ss.iqr(ac_uoi_srs) / 2))
print('UoI: %0.3f +/- %0.3f' %(uoi_median, ss.iqr(ac_baseline_srs) / 2))
print('Reduction Factor: ', baseline_median / uoi_median)
print('p-value: ', ss.wilcoxon(ac_baseline_srs, ac_uoi_srs, alternative='greater').pvalue)
print("Cohen's d: ", cohens_d(ac_baseline_srs, ac_uoi_srs))

AC
--
Baseline: 0.625 +/- 0.114
UoI: 0.250 +/- 0.116
Reduction Factor:  2.5
p-value:  6.825887996701619e-23
Cohen's d:  2.1911714273881033


## Predictive Performance

In [8]:
print('Retina')
print('------')
baseline_median = np.median(ret_baseline_r2s)
uoi_median = np.median(ret_uoi_r2s)

print('Baseline: %0.3f +/- %0.3f' %(baseline_median, ss.iqr(ret_baseline_r2s) / 2))
print('UoI: %0.3f +/- %0.3f' %(uoi_median, ss.iqr(ret_uoi_r2s) / 2))
print('Median Difference: ', np.median(baseline_median - uoi_median))
print('p-value: ', ss.wilcoxon(ret_baseline_r2s, ret_uoi_r2s, alternative='less').pvalue)
print("Cohen's d: ", cohens_d(ret_baseline_r2s, ret_uoi_r2s))

Retina
------
Baseline: 0.028 +/- 0.013
UoI: 0.028 +/- 0.013
Median Difference:  -0.0005507204601976623
p-value:  0.0037196019658107394
Cohen's d:  -0.04859165756986666


In [9]:
print('AC')
print('--')
baseline_median = np.median(ac_baseline_r2s)
uoi_median = np.median(ac_uoi_r2s)

print('Baseline: %0.3f +/- %0.3f' %(baseline_median, ss.iqr(ac_baseline_r2s) / 2))
print('UoI: %0.3f +/- %0.3f' %(uoi_median, ss.iqr(ac_uoi_r2s) / 2))
print('Median Delta: ', np.median(baseline_median - uoi_median))
print('p-value: ', ss.wilcoxon(ac_baseline_r2s, ac_uoi_r2s, alternative='greater').pvalue)
print("Cohen's d: ", cohens_d(ac_baseline_r2s, ac_uoi_r2s))

AC
--
Baseline: 0.042 +/- 0.034
UoI: 0.041 +/- 0.033
Median Delta:  0.00144526528523177
p-value:  1.8001739881993552e-19
Cohen's d:  0.04634772011377138


## Bayesian Information Criterion

In [10]:
print('Retina')
print('------')
baseline_median = np.median(ret_baseline_bics)
uoi_median = np.median(ret_uoi_bics)

print('Baseline: %0.3f +/- %0.3f' %(baseline_median, ss.iqr(ret_baseline_bics) / 2))
print('UoI: %0.3f +/- %0.3f' %(uoi_median, ss.iqr(ret_uoi_bics) / 2))
print('Median Delta: ', np.median(baseline_median - uoi_median))

Retina
------
Baseline: -1646606.843 +/- 44936.489
UoI: -1647261.080 +/- 44857.249
Median Delta:  654.237336314749


In [11]:
print('AC')
print('--')
baseline_median = np.median(ac_baseline_bics)
uoi_median = np.median(ac_uoi_bics)

print('Baseline: %0.3f +/- %0.3f' %(baseline_median, ss.iqr(ac_baseline_bics) / 2))
print('UoI: %0.3f +/- %0.3f' %(uoi_median, ss.iqr(ac_uoi_bics) / 2))
print('Median Delta: ', np.median(baseline_median - uoi_median))

AC
--
Baseline: 4370.767 +/- 2495.118
UoI: 4351.323 +/- 2493.153
Median Delta:  19.444203214014124
